In [3]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [11]:
df = pd.read_csv("res/smol_features.csv")
df = df.sort_values(by="Members", ascending =False, kind='quicksort', na_position='last', ignore_index=False, key=None)
df.head()

,Unnamed: 0,MAL_ID,Episodes,Duration,Members,Genres_Action,Genres_Adventure,Genres_Cars,Genres_Comedy,Genres_Dementia,...,Source_Visual novel,Source_Web manga,Rating_G,Rating_PG,Rating_PG13,Rating_R,Rating_R+,Rating_Rx,Aired_Start,Aired_End
3024,7449,16498,25.0,24.0,2531397,1,0,0,0,0,...,0,0,0,0,0,1,0,0,2013-04-07,2013-09-29
3181,8646,22319,12.0,24.0,1895488,1,0,0,0,0,...,0,0,0,0,0,1,0,0,2014-07-04,2014-09-19
1920,6474,11061,148.0,23.0,1673924,1,1,0,0,0,...,0,0,0,0,1,0,0,0,2011-10-02,2014-09-24
1529,8883,23273,22.0,22.0,1442330,0,0,0,0,0,...,0,0,0,0,1,0,0,0,2014-10-10,2015-03-20
867,8551,21881,24.0,23.0,1437210,1,1,0,0,0,...,0,0,0,0,1,0,0,0,2014-07-05,2014-12-20


In [39]:
series1 = []
series2 = []
for column in df:
    if "Genres" in str(column):
        sum1 = df.loc[df[column] == 1, 'Members'].sum()
        series1.append(column)
        series2.append(sum1)
df_genres_sum = pd.DataFrame({"Genre":series1, "Sum_Genre":series2})
df_genres_sum

,Genre,Sum_Genre
0,Genres_Action,47441235
1,Genres_Adventure,22741572
2,Genres_Cars,173483
3,Genres_Comedy,49470549
4,Genres_Dementia,2040856
5,Genres_Demons,7759396
6,Genres_Drama,35136777
7,Genres_Ecchi,10800346
8,Genres_Fantasy,30043771
9,Genres_Game,7164129


In [42]:
import plotly.express as px

fig = px.pie(df_genres_sum, values='Sum_Genre', names='Genre', title='Different Genre popularity')
fig.show()

## Data Wrangling

In [19]:
pd.set_option('display.max_columns', None)
features = df

In [20]:
#convert list columns into individual sections
def split_list(column):
    global features
    features[column] = features[column].apply(lambda x: [_.strip() for _ in x.split(",")])


split_list("Genres")
split_list("Producers")
split_list("Licensors")
split_list("Studios")

In [21]:
#replace unknowns with np.nan
features.replace(["Unknown", None], np.nan, inplace=True)

In [22]:
#convert strings to datetime objects
def str_to_dt(string):
    string = str(string)
    string_list = [string]
    if "to" in string:
        string_list = string.split("to")
        
    dts = []
    for i in string_list:
        i = i.strip()
        if i=="?" or i=="nan":
            string_dt = np.nan 
        else:
            try:
                string_dt = dt.strptime(i, "%b %d, %Y")
            except:
                try:
                    string_dt = dt.strptime(i, "%b, %Y")
                except:
                    string_dt = dt.strptime(i, "%Y")
        dts.append(string_dt)
    return dts

features["Aired"] = features["Aired"].apply(str_to_dt)

In [23]:
#convert duration into int
def str_to_num(string):
    
    string = str(string)
    if "nan" in string:
        return np.nan
    
    string_lst = string.split(".")
    minute = 0
    hour = 0
    
    for i in string_lst:
        if "min" in i:
            minute += int(i.strip("min "))
        if "hr" in i:
            hour += int(i.strip("hr "))
    
    return minute+(60*hour)

features["Duration"] = features["Duration"].apply(str_to_num)

In [24]:
#simplify Rating
def simplify_rating(rating):
    
    rating = str(rating)
    if "nan" in rating:
        return np.nan
    
    if rating.startswith("R"):
        if rating.startswith("Rx"):
            return "Rx"
        if rating.startswith("R+"):
            return "R+"
        return "R"
    
    if rating.startswith("PG"):
        if "13" in rating:
            return"PG13"
        return "PG"
    
    if rating.startswith("G"):
        return "G"

features["Rating"] = features["Rating"].apply(simplify_rating)

In [25]:
features.head()

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"[Action, Adventure, Comedy, Drama, Sci-Fi, Space]",Cowboy Bebop,カウボーイビバップ,TV,26,"[1998-04-03 00:00:00, 1999-04-24 00:00:00]",Spring 1998,[Bandai Visual],"[Funimation, Bandai Entertainment]",[Sunrise],Original,24.0,R,28.0,39,1251960,61971,105808,718161,71513,26678,329800,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"[Action, Drama, Mystery, Sci-Fi, Space]",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,[2001-09-01 00:00:00],NaN,"[Sunrise, Bandai Visual]",[Sony Pictures Entertainment],[Bones],Original,115.0,R,159.0,518,273145,1174,4143,208333,1935,770,57964,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,6,Trigun,8.24,"[Action, Sci-Fi, Adventure, Comedy, Drama, Sho...",Trigun,トライガン,TV,26,"[1998-04-01 00:00:00, 1998-09-30 00:00:00]",Spring 1998,[Victor Entertainment],"[Funimation, Geneon Entertainment USA]",[Madhouse],Manga,24.0,PG13,266.0,201,558913,12944,29113,343492,25465,13925,146918,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,7,Witch Hunter Robin,7.27,"[Action, Mystery, Police, Supernatural, Drama,...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"[2002-07-02 00:00:00, 2002-12-24 00:00:00]",Summer 2002,"[TV Tokyo, Bandai Visual, Dentsu, Victor Enter...","[Funimation, Bandai Entertainment]",[Sunrise],Original,25.0,PG13,2481.0,1467,94683,587,4300,46165,5121,5378,33719,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
4,8,Bouken Ou Beet,6.98,"[Adventure, Fantasy, Shounen, Supernatural]",Beet the Vandel Buster,冒険王ビィト,TV,52,"[2004-09-30 00:00:00, 2005-09-29 00:00:00]",Fall 2004,"[TV Tokyo, Dentsu]",[Unknown],[Toei Animation],Manga,23.0,PG,3710.0,4369,13224,18,642,7314,766,1108,3394,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0
